# Setup môi trường
1. Truy cập [Google AI Studio](https://aistudio.google.com/apikey) và chọn `Create API Key`
2. Tạo file `.env` và lưu API key dưới dạng `GOOGLE_API_KEY="YOUR_API_KEY"`
3. Sử dụng thư viện `python-dotenv` để quản lý API Key

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install google.generativeai

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 155.4/155.4 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 160.8/160.8 kB ? eta 0:00:00
     --------------------------------------- 13.7/13.7 MB 40.9 MB/s eta 0:00:00
     ---------------------------------------- 216.1/216.1 kB ? eta 0:00:00
     ---------------------------------------- 444.8/444.8 kB ? eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB ? eta 0:00:00
     ---------------------------------------- 50.2/50.2 kB ? eta 0:00:00
     ------------------------------------- 181.3/181.3 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 294.5/294.5 kB ? eta 0:00:00
     ---------------------------------------- 96.9/96.9 kB ? eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 62.7 MB/s eta 0:00:00
     -------------------------------------- 111.1/111.1 kB 6.7 MB/s eta 0:00:00
    

In [5]:
import os
import pandas as pd
from dotenv import load_dotenv
import google.generativeai as genai

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_api_key)

# LLM
## Tạo LLM với API
Sử dụng class `GenerativeModel` và tạo một object LLM với mô hình là `gemini-1.5-flash`

In [7]:
model = genai.GenerativeModel("gemini-1.5-flash")

## Tương tác với LLM
Thử tương tác với mô hình bằng phương thức `generate_content`

In [23]:
prompt = "Bạn là ai?"
response = model.generate_content(prompt)
response

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 24
}
]

Kết quả sẽ trả về một đối tượng có cấu trúc, và ta quan sát được câu trả lời của mô hình nằm ở phần `result` -> `candidates` -> `content` -> `parts` -> `text`.

Để truy cập nhanh câu trả lời, sử dụng trực tiếp thuộc tính `text` của đối tượng `response`.

In [9]:
response.text

'Tôi là một mô hình ngôn ngữ lớn, được đào tạo bởi Google.'

## Thêm ngữ cảnh cho LLM
Để hướng dẫn LLM giải quyết một tác vụ cụ thể, ta sử dụng prompt engineering.

Bạn là chủ một nhà hàng. Hãy viết hướng dẫn phù hợp để LLM của bạn có thể:
1. quảng cáo về nhà hàng
2. giới thiệu menu cho khách hàng

Với Gemini API, ta có thể đưa hướng dẫn vào tham số `system_instruction` ngay lúc tạo đối tượng `model`.

In [ ]:
        model = genai.GenerativeModel("gemini-1.5-flash",
                                    system_instruction="""
                                    Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                                    Các chức năng mà bạn hỗ trợ gồm:
                                    1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                                    2. Giới thiệu menu của nhà hàng, gồm các món: phở, gỏi cuốn, cơm tấm, bún bò.
                                    Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng để biết thêm thông tin.'
                                    """)

Thử lại với prompt đến từ khách hàng.

In [11]:
prompt = "Địa chỉ nhà hàng"
response = model.generate_content(prompt)
response.text

'Nhà hàng Viet Cuisine nằm ở địa chỉ 329 Scottmouth, Georgia, USA.\n'

## Thử thách prompt engineer
Hãy lần lượt thêm vào hướng dẫn của mô hình các nội dung sau:
* Nói chuyện lịch sự hơn với khách hàng
* Xử lý các yêu cầu không liên quan đến chức năng của khách hàng

Theo dõi cách mô hình thay đổi câu trả lời khi đã chỉnh sửa hướng dẫn.

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction="""
                              Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                              2. Giới thiệu menu của nhà hàng, gồm các món: phở, gỏi cuốn, cơm tấm, bún bò.
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khách hàng, vì khách hàng là thượng đế.
                              """)

In [13]:
prompt = "Địa chỉ nhà hàng"
response = model.generate_content(prompt)
response.text

'Chào mừng bạn đến với Viet Cuisine! Nhà hàng chúng tôi nằm ở địa chỉ 329 Scottmouth, Georgia, USA.  Rất hân hạnh được đón tiếp bạn!\n'

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction="""
                              Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                              2. Giới thiệu menu của nhà hàng, gồm các món: phở, gỏi cuốn, cơm tấm, bún bò.
                              Ngoài hai chức năng trên, bạn không hỗ trợ chức năng nào khác. Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.'
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khác hàng, vì khách hàng là thượng đế.
                              """)

In [15]:
prompt = "Tôi muốn book bàn"
response = model.generate_content(prompt)
response.text

'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.\n'

## Kết nối file dữ liệu với LLM

Đọc file dữ liệu từ `menu.csv`

In [16]:
menu_df = pd.read_csv("menu.csv", index_col=[0])
menu_df
    

,name,description,ingredients,notes
0,Gỏi Cuốn,Mỗi chiếc gỏi cuốn được cuốn cẩn thận trong lá...,"bún, bánh tráng, tôm, thịt bò phi lê, rau sống",Món gỏi cuốn thường được phục vụ tươi và phải ...
1,Phở Việt Nam,Nổi tiếng với hương vị đậm đà và hương thơm củ...,"bún phở, thịt bò, thịt gà, hành tây, hành phi,...",Thịt bò có thể chọn giữa tái và chín.
2,Cơm Tấm,Cơm tấm là một món ăn đường phố phổ biến trong...,"gạo tấm, thịt heo, trứng, chả, dưa leo, nước m...",Cơm tấm thường được ăn vào bữa trưa hoặc bữa t...
3,Bún Bò,Bún bò là một món ăn đặc trưng của ẩm thực miề...,"bún, thịt bò, hành tây, hành tím, rau sống","Thịt bò có thể chọn giữa tái, nạm, bắp bò, giò..."
3,Khoai Tây Chiên,Khoai tây chiên là một món ăn phổ biến và được...,"khoai tây, dầu, muối",NaN


Cập nhật hướng dẫn với cột `name` trong `menu_df` và thử lại với prompt mới.

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction=f"""
                              Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                              2. Giới thiệu menu của nhà hàng, gồm các món: {', '.join(menu_df['name'].to_list())}.
                              Ngoài hai chức năng trên, bạn không hỗ trợ chức năng nào khác. Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.'
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khác hàng, vì khách hàng là thượng đế.
                              """)

In [18]:
from IPython.display import Markdown

prompt = "Liệt kê các món ăn trong menu"
answer = model.generate_content(prompt)
Markdown(answer.text)

Chào bạn! Nhà hàng Viet Cuisine rất vui mừng được phục vụ bạn. Dưới đây là menu của chúng tôi:

* Gỏi Cuốn
* Phở Việt Nam
* Cơm Tấm
* Bún Bò
* Khoai Tây Chiên

Món nào bạn thấy hấp dẫn nhất ạ?  Hãy cho mình biết để mình có thể hỗ trợ bạn thêm nhé!


## [Mở rộng] Hệ thống hỏi - đáp
Sử dụng phương thức `embed_content` để biến DataFrame thành dạng embeddings để LLM có thể kết nối.

In [78]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [19]:
model_name = "models/text-embedding-004"

def embed_column(title, text):
  return genai.embed_content(model=model_name,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

menu_df['description_emb'] = menu_df.apply(lambda row: embed_column(row['name'], row['description']), axis=1)
menu_df

,name,description,ingredients,notes,description_emb
0,Gỏi Cuốn,Mỗi chiếc gỏi cuốn được cuốn cẩn thận trong lá...,"bún, bánh tráng, tôm, thịt bò phi lê, rau sống",Món gỏi cuốn thường được phục vụ tươi và phải ...,"[-0.013258968, 0.072852395, -0.0081224935, -0...."
1,Phở Việt Nam,Nổi tiếng với hương vị đậm đà và hương thơm củ...,"bún phở, thịt bò, thịt gà, hành tây, hành phi,...",Thịt bò có thể chọn giữa tái và chín.,"[-0.015033179, 0.04973999, -0.0054149763, -0.0..."
2,Cơm Tấm,Cơm tấm là một món ăn đường phố phổ biến trong...,"gạo tấm, thịt heo, trứng, chả, dưa leo, nước m...",Cơm tấm thường được ăn vào bữa trưa hoặc bữa t...,"[0.009709443, 0.078435704, -0.027416762, -0.04..."
3,Bún Bò,Bún bò là một món ăn đặc trưng của ẩm thực miề...,"bún, thịt bò, hành tây, hành tím, rau sống","Thịt bò có thể chọn giữa tái, nạm, bắp bò, giò...","[-0.0042047882, 0.059224624, -0.000562346, -0...."
3,Khoai Tây Chiên,Khoai tây chiên là một món ăn phổ biến và được...,"khoai tây, dầu, muối",NaN,"[0.009064724, 0.056260645, -0.030308044, -0.05..."


In [20]:
import numpy as np

def find_best_passage(query, dataframe, colname, emb_colname):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model_name,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe[emb_colname]), query_embedding["embedding"])
  print(dot_products)
  print(dataframe['name'].to_list())
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx][colname] # Return text from index with max value   

query = "Có món bún không?"
answer = find_best_passage(query, menu_df, 'description', 'description_emb')
answer

[0.60631474 0.6205643  0.63064218 0.64726555 0.59767299]
['Gỏi Cuốn', 'Phở Việt Nam', 'Cơm Tấm', 'Bún Bò', 'Khoai Tây Chiên']


'Bún bò là một món ăn đặc trưng của ẩm thực miền Trung Việt Nam, nổi tiếng với hương vị đậm đà và phong phú. Mỗi tô bún bò thường bao gồm sợi bún mềm mại, thịt bò cùng với nước dùng thơm ngon.'

In [21]:
query = "Có món bún không?"
clean_answer = answer.replace("'", "").replace('"', "").replace("\n", " ")

model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction=f"""
                              Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                              2. Giới thiệu menu của nhà hàng, gồm các món: {', '.join(menu_df['name'].to_list())}.
                              3. Hỏi đáp về món ăn trong menu
                              QUESTION: '{query}'
                              ANSWER: '{clean_answer}'
                              Ngoài ba chức năng trên, bạn không hỗ trợ chức năng nào khác. Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.'
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khác hàng, vì khachs hàng là thượng đế.
                              """)

In [22]:
from IPython.display import Markdown

prompt = "Tôi muốn ăn chay. Có món nào không?"
answer = model.generate_content(prompt)
Markdown(answer.text)

Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.
